In [1]:
import unittest
import pandas as pd
import os.path
from pathlib import Path

# Make sure we have a data directory
Path('./test_data').mkdir(parents=True, exist_ok=True) 

# Test Harness

In [2]:
import pandas_datareader as pdr
from datetime import datetime

test_data_file = 'test_data/test_data.csv'
if os.path.exists(test_data_file):
    test_data_df = pd.read_csv(test_data_file, header=[0, 1], index_col=[0], low_memory=False)
else:
    start = datetime(year=2019, month=1, day=1)
    end = datetime(year=2020, month=1, day=1)

    yahoo_reader = pdr.yahoo.daily.YahooDailyReader(symbols=['AAPL', 'GOOG'], start=start, end=end, adjust_price=True, interval='d', get_actions=False, adjust_dividends=True)
    test_data_df = yahoo_reader.read()
    yahoo_reader.close()
    test_data_df.to_csv(test_data_file, index=True)
    

test_snp_500_stocks_file = 'test_data/snp500.csv'
if os.path.exists(test_snp_500_stocks_file):
    snp_500_stocks = pd.read_csv(test_snp_500_stocks_file, index_col=[0], low_memory=False)
else:
    snp_500_stocks = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', 
                            header=0, 
                            attrs={'id': 'constituents'}, 
                            index_col='Symbol')[0]
    snp_500_stocks.to_csv(test_snp_500_stocks_file)

In [3]:
test_data_df.head()

Attributes Adj_Ratio           Close                    High               \
Symbols         AAPL GOOG       AAPL         GOOG       AAPL         GOOG   
Date                                                                        
2019-01-02  0.969542  1.0  38.277523  1045.849976  38.502944  1052.319946   
2019-01-03  0.969542  1.0  34.464794  1016.059998  35.320415  1056.979980   
2019-01-04  0.969542  1.0  35.936081  1070.709961  36.006375  1070.839966   
2019-01-07  0.969542  1.0  35.856091  1068.390015  36.074240  1074.000000   
2019-01-08  0.969542  1.0  36.539616  1076.280029  36.798970  1084.560059   

Attributes        Low                    Open                    Volume  \
Symbols          AAPL         GOOG       AAPL         GOOG         AAPL   
Date                                                                      
2019-01-02  37.383120  1015.710022  37.543095  1016.570007  148158800.0   
2019-01-03  34.418740  1014.070007  34.898663  1041.000000  365248800.0   
2019-01-04  34.855043  1027.417969  35.031984  1032.589966  234428400.0   
2019-01-07  35.364048  1054.760010  36.042728  1071.500000  219111200.0   
2019-01-08  35.999097  1060.530029  36.251176  1076.109985  164101200.0   

Attributes             
Symbols          GOOG  
Date                   
2019-01-02  1532600.0  
2019-01-03  1841100.0  
2019-01-04  2093900.0  
2019-01-07  1981900.0  
2019-01-08  1764900.0

In [4]:
test_data_df.tail()

Attributes Adj_Ratio           Close                    High               \
Symbols         AAPL GOOG       AAPL         GOOG       AAPL         GOOG   
Date                                                                        
2019-12-24  0.984110  1.0  69.938202  1343.560059  70.090745  1350.260010   
2019-12-26  0.984109  1.0  71.325790  1360.400024  71.343014  1361.327026   
2019-12-27  0.984110  1.0  71.298744  1351.890015  72.324682  1364.530029   
2019-12-30  0.984110  1.0  71.721901  1336.140015  72.009756  1353.000000   
2019-12-31  0.984110  1.0  72.245964  1337.020020  72.253345  1338.000000   

Attributes        Low                    Open                    Volume  \
Symbols          AAPL         GOOG       AAPL         GOOG         AAPL   
Date                                                                      
2019-12-24  69.606071  1342.780029  70.041537  1348.500000   48478800.0   
2019-12-26  70.043990  1344.469971  70.073512  1346.170044   93121200.0   
2019-12-27  70.885420  1349.310059  71.623502  1362.989990  146266000.0   
2019-12-30  70.171931  1334.020020  71.215085  1350.000000  144114400.0   
2019-12-31  71.229869  1329.084961  71.330742  1330.109985  100805600.0   

Attributes             
Symbols          GOOG  
Date                   
2019-12-24   347500.0  
2019-12-26   667500.0  
2019-12-27  1038400.0  
2019-12-30  1050900.0  
2019-12-31   961800.0

In [5]:
test_data_df['Close'].head()

Symbols,AAPL,GOOG
Date,,
2019-01-02,38.277523,1045.849976
2019-01-03,34.464794,1016.059998
2019-01-04,35.936081,1070.709961
2019-01-07,35.856091,1068.390015
2019-01-08,36.539616,1076.280029


In [6]:
test_data_df['Close'].tail()

Symbols,AAPL,GOOG
Date,,
2019-12-24,69.938202,1343.560059
2019-12-26,71.325790,1360.400024
2019-12-27,71.298744,1351.890015
2019-12-30,71.721901,1336.140015
2019-12-31,72.245964,1337.020020


# Unit Tests

In [7]:
import importlib
import trading_factors_yahoo as alpha_factors
importlib.reload(alpha_factors)

class TestFactorData(unittest.TestCase):
    def test_init(self):
        class_under_test = alpha_factors.FactorData(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'Alpha Factor')
        self.assertEqual(len(class_under_test.factor_data.columns), len(test_data_df.columns))
        
    def test_rank(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df).rank()
        self.assertEqual(class_under_test.factor_data.loc['2019-01-03']['AAPL'], 1.0)
        self.assertEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 2.0)
    
    def test_demean(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df).demean()
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-03']['AAPL'], -0.035561, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 0.0033238, places=4)
        
    def test_zscore(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df).zscore()
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-03']['AAPL'], -1.0, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 1.0, places=4)
        
    def test_smoothed(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df).smoothed(2)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-04']['AAPL'], -0.028459, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 0.006621, places=4)
        
    def test_for_al(self):
        series_data = alpha_factors.FactorReturns(test_data_df).for_al()
        self.assertEqual(series_data.name, 'returns_1_day')
        self.assertTupleEqual(series_data.shape, (502,))
        self.assertAlmostEqual(series_data.loc['2019-12-31', 'AAPL'], 0.007306, places=4)
        
    def test_open_values(self):
        class_under_test = alpha_factors.OpenPrices(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'open')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-02']['AAPL'], 37.543095, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 71.330711, places=4)

    def test_close_values(self):
        class_under_test = alpha_factors.ClosePrices(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'close')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-02']['AAPL'], 38.277523, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 72.245934, places=4)
        
    def test_volume_values(self):
        class_under_test = alpha_factors.Volume(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'volume')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-02']['AAPL'], 148158800.0, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 100805600.0, places=4)
        
    def test_top(self):
        class_under_test = alpha_factors.Volume(test_data_df).top(1)
        self.assertEqual(len(class_under_test), 1)
        self.assertEqual(class_under_test[0], 'AAPL')
        
    def test_daily_dollar_volume(self):
        class_under_test = alpha_factors.DailyDollarVolume(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'daily_dollar_volume')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-02']['AAPL'], 5671151880.6930, places=2)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 7282797753.668213, places=2)
        
    def test_average_dollar_volume(self):
        class_under_test = alpha_factors.AverageDollarVolume(test_data_df, 5)
        self.assertEqual(class_under_test.factor_name, 'average_dollar_volume_5_day')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-08']['AAPL'], 8107296068.833008, places=2)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 7616000377.970581, places=2)
        
    def test_returns(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'returns_1_day')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (252,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 0.007306, places=4)
        
    def test_momentum(self):
        class_under_test = alpha_factors.FactorMomentum(test_data_df, 5)
        self.assertEqual(class_under_test.factor_name, 'momentum_5_day')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (252,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 0.033978, places=4)    
        
    def test_meanreversion(self):
        class_under_test = alpha_factors.FactorMeanReversion(test_data_df, 5)
        self.assertEqual(class_under_test.factor_name, 'mean_reversion_5_day_logret')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (252,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], -0.033978, places=4)

    def test_close_to_open(self):
        class_under_test = alpha_factors.CloseToOpen(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'close_to_open')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (252,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-30']['AAPL'], -0.005454, places=4)
        
    def test_trailing_overnight_returns(self):
        class_under_test = alpha_factors.TrailingOvernightReturns(test_data_df, 10)
        self.assertEqual(class_under_test.factor_name, 'trailing_overnight_returns_10_day')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (252,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-30']['AAPL'], 0.00963419, places=4)
    
    def test_annualized_volatility(self):
        class_under_test = alpha_factors.AnnualizedVolatility(test_data_df, 20)
        self.assertEqual(class_under_test.factor_name, 'annualzed_volatility_20_day')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (213,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-30']['AAPL'], 0.3957571629968691, places=4)
        
    def test_market_dispersion(self):
        class_under_test = alpha_factors.MarketDispersion(test_data_df, 20)
        self.assertEqual(class_under_test.factor_name, 'market_dispersion_20_day')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (251,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-30']['AAPL'], 0.0042166016735185, places=4)
        
    def test_get_sector_helper(self):
        class_under_test = alpha_factors.get_sector_helper(snp_500_stocks, 'GICS Sector', ['AAPL', 'GOOG'])
        self.assertEqual(class_under_test['Communication Services'], ['GOOG'])
        self.assertEqual(class_under_test['Information Technology'], ['AAPL'])
        
    def test_filter_price_histories(self):
        class_under_test = set(alpha_factors.filter_price_histories(test_data_df, ['AAPL']).columns.get_level_values('Symbols').tolist())
        self.assertEqual(class_under_test.pop(), 'AAPL')


In [8]:
TestFactorData().test_annualized_volatility()

In [9]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_annualized_volatility (__main__.TestFactorData) ... ok
test_average_dollar_volume (__main__.TestFactorData) ... ok
test_close_to_open (__main__.TestFactorData) ... ok
test_close_values (__main__.TestFactorData) ... ok
test_daily_dollar_volume (__main__.TestFactorData) ... ok
test_demean (__main__.TestFactorData) ... ok
test_filter_price_histories (__main__.TestFactorData) ... ok
test_for_al (__main__.TestFactorData) ... ok
test_get_sector_helper (__main__.TestFactorData) ... ok
test_init (__main__.TestFactorData) ... ok
test_market_dispersion (__main__.TestFactorData) ... ok
test_meanreversion (__main__.TestFactorData) ... ok
test_momentum (__main__.TestFactorData) ... ok
test_open_values (__main__.TestFactorData) ... ok
test_rank (__main__.TestFactorData) ... ok
test_returns (__main__.TestFactorData) ... ok
test_smoothed (__main__.TestFactorData) ... ok
test_top (__main__.TestFactorData) ... ok
test_trailing_overnight_returns (__main__.TestFactorData) ... ok
test_volume_values (_